In [28]:
#Импортирование библиотек и подготовка исходных файлов
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

In [29]:

def stud_df_func(file_name):
    
    if file_name == 'datastud.xlsx' :
        stud_df = pd.ExcelFile(file_name)
        table = stud_df.parse('Результаты')
        t_stud = table.copy()
        t_stud = t_stud.iloc[:, [0,1,2,3,11]] 
    elif file_name == 't_stud_new.xlsx' :
        stud_df = pd.ExcelFile(file_name)
        table = stud_df.parse('Результаты')
        t_stud = table.copy()
    
    if set(['Unnamed: 0']).issubset(t_stud.columns):
        t_stud.drop('Unnamed: 0', axis = 1, inplace = True)
        
    return t_stud

In [30]:
#Ранжирование
def ranking(t_stud):
    
    flag_rank = True
    
    for a in t_stud['Группа'].unique().tolist():
        for b in t_stud['Студент (Код)'].unique().tolist():
            for c in t_stud['Семестр'].unique().tolist():
                t_stud_1 = t_stud.loc[(t_stud['Студент (Код)'] == b) & (t_stud['Группа'] == a) & (t_stud['Семестр'] == c)]
                t_stud_1['Ранг'] = t_stud_1['Оценка студента (опрос)'].rank(method='dense', ascending=False)
                if flag_rank == True:
                    flag_rank = False
                    empty_df_stud = t_stud_1.iloc[0:0]
                    global_df_stud = pd.concat([t_stud_1, empty_df_stud])
                else:
                    global_df_stud = pd.concat([t_stud_1, global_df_stud])

    rank_st = global_df_stud
    
    return rank_st

In [31]:
#строим словари для каждого ранжирования
#Строим бинарные матрицы отношений рангов
#Строим матрицы отношений
#rdict = {}
def array_of_disc(rank_st):
    
    disc_array = rank_st['Дисциплина'].unique().tolist()
    disc_array.sort()

    return disc_array

#bin_matrix_plus = np.zeros(shape=(len(disc_array),len(disc_array)))
#bin_matrix_zeros = np.zeros(shape=(len(disc_array),len(disc_array)))
#bin_matrix_minus = np.zeros(shape=(len(disc_array),len(disc_array)))

#rank_array_dict = {}
#dictionary_of_disc = {}
#dictionary_of_rank = {}
#count_rang = 0

def rank_disc_dict(rank_st):
    
    
    global bin_matrix_plus
    global bin_matrix_zeros
    global bin_matrix_minus
    global rank_array_dict
    global dictionary_of_disc
    global dictionary_of_rank
    global count_rang

    s = 0
    
    for k in rank_st['Группа'].unique().tolist():
        for g in rank_st['Студент (Код)'].unique().tolist():
            for d in rank_st['Семестр'].unique().tolist():
                rnump_w_na = rank_st.loc[(rank_st['Студент (Код)'] == g) & (rank_st['Группа'] == k) & (rank_st['Семестр'] == d)]['Ранг']
                rnump_notclean = rank_st.loc[(rank_st['Студент (Код)'] == g) & (rank_st['Группа'] == k) & (rank_st['Семестр'] == d)]['Ранг'].to_numpy()
                bin_disc = rank_st.loc[(rank_st['Студент (Код)'] == g) & (rank_st['Группа'] == k) & (rank_st['Семестр'] == d)]['Дисциплина'].to_numpy()
                count_one = 0
                count_nan = 0
                for ind in range(0, len(rnump_notclean)):
                    if rnump_notclean[ind] == 1:
                        count_one += 1
                    if np.isnan(rnump_notclean[ind]) == True:
                        count_nan += 1       
                if (count_one + count_nan) == len(rnump_notclean):
                    continue
                rank_disc_dict = []
                if count_one >= 1:
                    count_rang += 1
                    s += rnump_w_na.isna().sum()
                    rnump_w_na.fillna(value = -1, inplace = True)
                    rnump = rnump_w_na.to_numpy()
                    for i in range(0, len(rnump)):
                        for j in range(0, len(rnump)):
                            if (rnump[i] != -1) and (rnump[j] != -1):
                                if rnump[i] < rnump[j]:
                                    #rlist[bin_disc[i] - 1][bin_disc[j] - 1] = 1
                                    bin_matrix_plus[bin_disc[i] - 1][bin_disc[j] - 1] = bin_matrix_plus[bin_disc[i] - 1][bin_disc[j] - 1] + 1
                                if rnump[i] > rnump[j]:
                                    #rlist[bin_disc[i] - 1][bin_disc[j] - 1] = -1
                                    bin_matrix_minus[bin_disc[i] - 1][bin_disc[j] - 1] = bin_matrix_minus[bin_disc[i] - 1][bin_disc[j] - 1] + 1
                                if rnump[i] == rnump[j]:
                                    #rlist[bin_disc[i] - 1][bin_disc[j] - 1] = 0 
                                    bin_matrix_zeros[bin_disc[i] - 1][bin_disc[j] - 1] = bin_matrix_zeros[bin_disc[i] - 1][bin_disc[j] - 1] + 1
                    bin_disc = tuple(bin_disc)
                    dictionary_of_disc.update({(str(d) + str(k) + str(g), bin_disc)})
                    rnumpi = tuple(rnump)
                    dictionary_of_rank.update({(str(d) + str(k) + str(g), rnumpi)})
                    for i in range(0, len(rnump)):
                        disc_rank = [bin_disc[i], rnump[i]]
                        rank_disc_dict.append(disc_rank)
                        rnumpy = tuple(map(tuple, rank_disc_dict))
                        rank_array_dict.update({(str(d) + str(k) + str(g), rnumpy)})


In [32]:
#считаем показатель Т
def pokazatel_T(rank_array_dict, dictionary_of_disc):
    
    T_krit = {}
    count_array = []
    count_dict = {}
    #matrix_T = np.zeros(shape=(len(rank_array_dict),len(rank_array_dict)))

    for k,v in rank_array_dict.items():
        T_array = []
        set_k = set(dictionary_of_disc[k])
        len_k = len(set_k)
        for k2,v2 in rank_array_dict.items():
            if k2 != k:
                T = 0
                disc_plus = 0
                simular_rank_count = 0
                count_zero_k = 0
                count_zero_k2 = 0
                set_k2 = set(dictionary_of_disc[k2])
                disc_minus = len(set_k.difference(set_k2))
                disc_pls = len(set_k.intersection(set_k2))
                k2_minus_k = len(set_k2.difference(set_k))

                for i in range(len(rank_array_dict[k])):
                    disc_k = rank_array_dict[k][i][0]
                    rank_k = rank_array_dict[k][i][1]
                    if rank_k == -1:
                        T += 0.5
                        count_zero_k += 1
                        continue
                    for j in range(len(rank_array_dict[k2])):
                        disc_k2 = rank_array_dict[k2][j][0]
                        rank_k2 = rank_array_dict[k2][j][1]   
                        if disc_k == disc_k2:
                            if rank_k2 == -1:
                                count_zero_k2 += 1
                                T += 0.5
                                continue

                            disc_plus += 1
                            T += (rank_k - rank_k2) ** 2
                            if rank_k == rank_k2:
                                simular_rank_count += 1
                            break
                
                T = T + disc_minus * 0.25

                T_kr = [k2, T, disc_pls, disc_minus, simular_rank_count, count_zero_k, count_zero_k2, len_k]
                T_array.append(T_kr)
                T_T = tuple(map(tuple, T_array))
                T_krit.update({(k, T_T)})
                
    return T_krit

In [33]:
#считаем близость ранжирований
#indexs = []

def d_matrix_step_1(T_krit):
    
    t_const = 2
    global indexs
    matrix_1_step = np.zeros(shape=(len(T_krit),len(T_krit)))
    
    for key0,values0 in T_krit.items():
        indexs.append(key0)
        
    step_1_df = pd.DataFrame(matrix_1_step, index = indexs, columns = indexs)

    for key, value in T_krit.items():
        for f in range(len(T_krit[key])):
            describe_data = value[f][0]
            T_koef = value[f][1]
            simular_disc = value[f][2]
            diff_disc = value[f][3]
            simular_rank = value[f][4]
            leng_data = value[f][7]
            #  T_koef > t_const or simular_disc < 5 or diff_disc > 3 or key == describe_data or simular_disc/leng_data < 0.5
            if T_koef > t_const or simular_disc < 5 or diff_disc > 3 or key == describe_data:
                step_1_df[key][describe_data] = 0
            else: 
                step_1_df[key][describe_data] = 1
                
    return step_1_df

In [34]:
# step 2
def G_first_step_2(step_1_df):

    step_1_df['Counts_of_1'] = step_1_df.sum(axis=1)
    step_1_df_index = step_1_df.reset_index()
    maximum = step_1_df_index['Counts_of_1'].max()
    #pd.options.display.max_rows = 50
    #max_step_1 = step_1_df_index.loc[step_1_df_index['Counts_of_1'] == maximum]
    #max_step_1
    st_w_o_0 = step_1_df_index.loc[step_1_df_index['Counts_of_1'] != 0].shape
    st_w = (st_w_o_0[0])/10
    larg_20 = step_1_df_index.nlargest(round(st_w), columns = ['Counts_of_1'])
    #larg_20
    #for key, value in max_step_1.iteritems():

    T_shsh = larg_20['Counts_of_1'].min()
    G_first = larg_20['index'].to_numpy()
    
    return G_first


In [35]:
def d_sh_matrix(G_first):
    
    matrix_d_sh = np.zeros(shape=(len(G_first), len(G_first)))
    df_d_sh = pd.DataFrame(matrix_d_sh, index = G_first, columns = G_first)
    
    return df_d_sh

In [36]:
# step 3-4
def koef_d_sh(df_d_sh, G_first, indexs, step_1_df):
    
    d_sh = 0

    for ki in G_first:
        for kj in G_first: 
            ssum = 0
            for k in indexs:
                ssum += step_1_df[ki][k] * step_1_df[k][kj]
            d_sh = (2/(step_1_df['Counts_of_1'][ki] + step_1_df['Counts_of_1'][kj])) * ssum
            df_d_sh[ki][kj] = d_sh
    
    return df_d_sh
        


In [37]:
#step 6
def G_k_step_6(df_d_sh, G_first):
    
    T_sssh = 0.75
    G_k = []
    
    for ki in G_first:
        for kj in G_first:
            if df_d_sh[ki][kj] >= T_sssh:
                if kj not in G_k:
                    G_k.append(kj)
                    
    return G_k


In [38]:
# разбиваем на группы
def grouping_step_6_7(G_first, G_k, df_d_sh):
    
    T_sssh = 0.75
    G_glob = []
    G_new = []
    
    if len(list(set(G_first) - set(G_k))) > 0:
        while len(G_k) > 0:
            for kki in G_k:
                for kkj in G_k:
                    if df_d_sh[kki][kkj] >= T_sssh:
                        if kkj not in G_new:
                            G_new.append(kkj)
            G_to = list(G_new)
            G_glob.append(G_to)
            G_k = list(set(G_k) - set(G_new))
    else:
        print('одинаковые')
    
    return G_glob

In [39]:
def checking_process(G_glob, flag2):
    global length_of_loop
    length_of_loop += 1
    if (not G_glob) or (length_of_loop == 10):
        flag2 = False
    return flag2
    

In [40]:
def checking_process_2(G_glob, flag2):
    global length_of_loop
    length_of_loop += 1
    if (not G_glob) or (length_of_loop == 10):
        flag2 = False
    return flag2

In [41]:
# step 8
# для каждой группы заполняем последний шаг
def adding_rang_step_8(G_glob, indexs, step_1_df):
    
    G_G = []
    
    for s in range(len(G_glob)):
        G_G = G_glob[s]
        G_k_sh = []
        for vvi in G_G:
            for vvj in G_G:
                for v in indexs:
                    if step_1_df[vvi][v] == 1 or step_1_df[vvj][v] == 1:
                        if v not in G_k_sh and v not in G_G:
                            G_k_sh.append(v)          
        G_glob[s].extend(G_k_sh)   
    
    return G_glob


In [42]:
#делаем листы дисциплин

# discs_minus = []

def list_disc(G_glob, dictionary_of_disc, dictionary_of_rank):
    
    all_disc = []
    all_ranks = []
    discs = []
    global discs_minus
    
    for i in range(len(G_glob)):
        for kglob in G_glob[i]:
            info_rang = rank_array_dict[kglob]
            all_disc.extend(list(set(dictionary_of_disc[kglob]))) 
            all_ranks.extend(list(set(dictionary_of_rank[kglob])))
            for x in all_disc:
                if x not in discs:
                    discs.append(x)
    c = Counter(all_disc)  
    
    discs_old = discs
    discs = [i for i in discs if c[i] >= len(discs)/4]
    discs_minus = list(set(discs_old) - set(discs))
    
    return discs

In [43]:
# заполнение матриц отношений

#matrix_plus_K = np.zeros(shape=(len(disc_array),len(disc_array)))
#matrix_zeros_K = np.zeros(shape=(len(disc_array),len(disc_array)))
#matrix_minus_K = np.zeros(shape=(len(disc_array),len(disc_array)))

def matrix_otn_group(G_glob, rank_st):
    
    global matrix_plus_K
    global matrix_zeros_K
    global matrix_minus_K

    for g in range(len(G_glob)):
        for l in range(len(G_glob[g])):
            semestr_id = int(G_glob[g][l][0])
            group_id = int(G_glob[g][l][1])
            student_id = int(G_glob[g][l][2:])
            rank_g_na = rank_st.loc[(rank_st['Студент (Код)'] == student_id) & (rank_st['Группа'] == group_id) & (rank_st['Семестр'] == semestr_id)]['Ранг']
            rank_g_notclean = rank_st.loc[(rank_st['Студент (Код)'] == student_id) & (rank_st['Группа'] == group_id) & (rank_st['Семестр'] == semestr_id)]['Ранг'].to_numpy()
            disc_g = rank_st.loc[(rank_st['Студент (Код)'] == student_id) & (rank_st['Группа'] == group_id) & (rank_st['Семестр'] == semestr_id)]['Дисциплина'].to_numpy()

            rank_g_na.fillna(value = -1, inplace = True)
            rank_g = rank_g_na.to_numpy()
            for i in range(len(rank_g)):
                for j in range(len(rank_g)):
                    if (rank_g[i] != -1) and (rank_g[j] != -1):
                        if rank_g[i] < rank_g[j]:
                            matrix_plus_K[disc_g[i] - 1][disc_g[j] - 1] = matrix_plus_K[disc_g[i] - 1][disc_g[j] - 1] + 1
                        if rank_g[i] > rank_g[j]: 
                            matrix_minus_K[disc_g[i] - 1][disc_g[j] - 1] = matrix_minus_K[disc_g[i] - 1][disc_g[j] - 1] + 1
                        if rank_g[i] == rank_g[j]: 
                            matrix_zeros_K[disc_g[i] - 1][disc_g[j] - 1] = matrix_zeros_K[disc_g[i] - 1][disc_g[j] - 1] + 1

In [44]:
# считаем K_s
# считаем K_z и K_z0
# k_z = (2 * summa(N_+_ij, где i > j))/n * m(m-1)
# k_z_0 = (2 * summa(1, когда N_+_ij !=0, i>j))/m(m-1), где i > j

def K_s_z_z0_function(matrix_plus_K, matrix_zeros_K, discs, G_glob):
    
    n_sum = 0
    n_plus_count = 0
    a1 = []
    a2 = []
    N_plus_summa = 0
    
    global K_s 
    global K_z_
    global K_z_0
    
    n_ = len(G_glob[0])
    m_ = len(discs) 
    
    for i in discs:
        for j in discs:
            if (i != j) and (j not in a1) and (i not in a2):
                if matrix_plus_K[i-1][j-1] != 0:
                    n_plus_count += 1
                    N_minus_ij = matrix_plus_K[i-1][j-1] - matrix_plus_K[j-1][i-1]
                    N_plus_ij = matrix_plus_K[i-1][j-1] + matrix_zeros_K[i-1][j-1] + matrix_plus_K[j-1][i-1]
                    N_plus_summa += N_plus_ij
                    n_sum += abs(N_minus_ij) / N_plus_ij
                    a1.append(i)
                    a2.append(j)


    #n_sum = 1/2 * (n_sum)
    #n_plus_count = 1/2 * (n_plus_count)
    if n_plus_count != 0:
        K_s = (1/n_plus_count) * (n_sum)
    else:
        k_z_ = 0
        K_z_0 = 0
    
    if n_ > 0 and (m_ - 1) > 0:
        K_z_ = (2 * N_plus_summa) / (n_ * m_ * (m_ - 1))
        K_z_0 = (2 * n_plus_count) / (m_ * (m_- 1))
    else:
        k_z_ = 0
        K_z_0 = 0



In [45]:
# считаем K_st 
# Nsht_ij = Nij + 0.5 N0ij

def K_st_function(matrix_plus_K, matrix_zeros_K, discs):
    
    K_st = 0
    checks_1 = []
    checks_2 = []
    for i in discs:
        for j in discs:
            if (i != j):
                N_sh_ij = matrix_plus_K[i-1][j-1] + 0.5 * matrix_zeros_K[i-1][j-1]
                N_sh_ji = matrix_plus_K[j-1][i-1] + 0.5 * matrix_zeros_K[j-1][i-1]
                if (N_sh_ij != 0) or (N_sh_ji != 0):
                    checks_1.append([N_sh_ij, N_sh_ji])
                if N_sh_ij > N_sh_ji:
                    for k in discs:
                        if (k != i) and (k != j):
                            N_sh_ik = matrix_plus_K[i-1][k-1] + 0.5 * matrix_zeros_K[i-1][k-1]
                            N_sh_ki = matrix_plus_K[k-1][i-1] + 0.5 * matrix_zeros_K[k-1][i-1]
                            N_sh_kj = matrix_plus_K[k-1][j-1] + 0.5 * matrix_zeros_K[k-1][j-1]
                            N_sh_jk = matrix_plus_K[j-1][k-1] + 0.5 * matrix_zeros_K[j-1][k-1]
                            if (N_sh_ik != 0) or (N_sh_ki != 0) or (N_sh_kj != 0) or (N_sh_jk != 0):
                                checks_2.append([N_sh_ik, N_sh_ki, N_sh_kj, N_sh_jk])
                            if (N_sh_ik < N_sh_ki) and (N_sh_kj < N_sh_jk):
                                K_st += 1
    
    return K_st


In [46]:
def results_K_s_K_st(K_st, K_s, discs):
    
    maxK_st = len(discs) * (((len(discs)) ** 2) - 1) / 8
    K_st = K_st / maxK_st
    
    return K_st

In [47]:
# заполнение датафрейма данными из выбранных ранжирований
def dataframe_of_group(G_glob, rank_st, discs):
    
    new_df_pd = pd.DataFrame(data = np.zeros(shape=(len(discs),len(G_glob[0]))), index = discs, columns = G_glob[0])
    
    for g in range(len(G_glob)):
        for l in range(len(G_glob[g])):
            semestr_id = int(G_glob[g][l][0])
            group_id = int(G_glob[g][l][1])
            student_id = int(G_glob[g][l][2:])
            disciplines = rank_st.loc[(rank_st['Студент (Код)'] == student_id) & (rank_st['Группа'] == group_id) & (rank_st['Семестр'] == semestr_id)]['Дисциплина'].to_numpy()
            for t in disciplines:
                abc = rank_st.loc[(rank_st['Студент (Код)'] == student_id) & (rank_st['Группа'] == group_id) & (rank_st['Семестр'] == semestr_id)].loc[rank_st['Дисциплина'] == t]['Оценка студента (опрос)'].to_numpy()
                if np.isnan(abc) == False:
                    new_df_pd[G_glob[g][l]][t] = int(abc)
                else:
                    new_df_pd[G_glob[g][l]][t] = np.nan
    
    return new_df_pd

In [48]:
def isnull_any(new_df_pd):
    yi = new_df_pd.replace(0, np.nan)
    return yi.isna().any().any()

In [49]:
#new_df_pd_w_true_nan = new_df_pd.copy()
#%run rang.ipynb

def vosstanovlenie_prop(new_df_pd):
    
    new_df_pd = new_df_pd.replace(0, np.nan)
    new_df_pd = new_df_pd.reset_index()
    list_index = new_df_pd['index']
    new_df_pd.drop(columns = ['index'], inplace=True)

    Matr = getMatrix(new_df_pd)
    print(new_df_pd)
    #print(Matrix)
    rez= method_rows_sum_extension(Matr)
    print('\n строчные суммы по Чеботарёву\n',rez['x'])
    print('\n ошибка = ',rez['err'])

    # текущий рейтинг (среднеариф )
    #x_=np.array([ 40, 30, 20,  10, 0])
    x_=np.array(new_df_pd.mean(axis = 1, skipna = True))
    print('\n текущий рейтинг \n ',x_)
    # итоговая матрица парных сравнений 
    #t_=rang.getT(rez['x'])

    # Востановление пропусков
    dfrez= method_rang_complit(new_df_pd,x_,Matr)
    dfrez = dfrez.round(1)
    print('\n данные с востановленными пропусками\n',dfrez)
    print('\n ошибка = ',rez['err'])

    print('\n итоговый рейтинг \n ',x_)
    y_=np.array(dfrez.mean(axis = 1))
    
    dfrez.set_index(list_index, inplace = True)
    
    return dfrez


In [56]:
#t_stud_update
def t_stud_update_to_excel(t_stud, dfrez):
    
    global flag_true
    global flag_true_2
    
    if flag_true == True:
        flag_true = False
    else:
        stud_x = pd.ExcelFile('t_stud_separate.xlsx')
        table_x = stud_x.parse('Результаты')
        t_stud_x = table_x.copy()
        if set(['Unnamed: 0']).issubset(t_stud_x.columns):
            t_stud_x.drop('Unnamed: 0', axis = 1, inplace = True)
    
    t_stud_plus = t_stud.iloc[0:0]

    for key, value in dfrez.iteritems(): 
        disci = dfrez.index.to_numpy()
        semestr_id = int(key[0])
        group_id = int(key[1])
        student_id = int(key[2:])
        for i in range(len(disci)):
            t_stud_plus = t_stud_plus.append({'Дисциплина': disci[i], 'Группа': group_id, 'Студент (Код)': student_id, 'Семестр': semestr_id, 'Оценка студента (опрос)': value[disci[i]]}, ignore_index = True)
    
    t_stud_plus.astype(int)

    t_stud_renew_plus = pd.concat([t_stud, t_stud_plus], ignore_index = True).drop_duplicates(subset=['Дисциплина', 'Группа', 'Студент (Код)', 'Семестр'], keep='last')
    
    
    if flag_true_2 == True:
        flag_true_2 = False
        t_stud_plus.to_excel("t_stud_separate.xlsx", sheet_name='Результаты')
    else:
        t_stud_renew_separate = pd.concat([t_stud_x, t_stud_plus], ignore_index = True)
        t_stud_renew_separate.to_excel("t_stud_separate.xlsx", sheet_name='Результаты')       
    
    t_stud_renew_plus.to_excel("t_stud_new.xlsx", sheet_name='Результаты')
    


In [51]:
def adding_nans(G_glob, discs, new_df_pd):
    
    need_df = new_df_pd.replace(0, np.nan)
    global af
    for i in G_glob[0]:
        for j in discs:
            if np.isnan(need_df[i][j]) == False:
                x = random.randrange(3)
                if x > 1:
                    af.append([i, j, need_df[i][j]])
                    need_df[i][j] = np.nan
                    
    return need_df

In [52]:

def raznost_nans(vosst_df_w_add_nans, af, new_df_pd):
    
    global otsenki
    new_list = new_df_pd.reset_index()
    list_index = new_list['index']
    #vosst_df.set_index(list_index, inplace = True)
    vosst_df_w_add_nans.set_index(list_index, inplace = True)
    sum_razn = 0
    
    for i in range(len(af)):
        razn = (new_df_pd[af[i][0]][af[i][1]] - vosst_df_w_add_nans[af[i][0]][af[i][1]]) ** 2
        otsenki.append([razn, af[i][0], af[i][1]])
        sum_razn += razn
    
    return sum_razn

In [53]:
#4
#bin_matrix_plus = np.zeros(shape=(len(disc_array),len(disc_array)))
#bin_matrix_zeros = np.zeros(shape=(len(disc_array),len(disc_array)))
#bin_matrix_minus = np.zeros(shape=(len(disc_array),len(disc_array)))
#rank_array_dict = {}
#dictionary_of_disc = {}
#dictionary_of_rank = {}
#count_rang = 0

#6
#indexs = []

#13
#discs_minus = []

#14
#matrix_plus_K = np.zeros(shape=(len(disc_array),len(disc_array)))
#matrix_zeros_K = np.zeros(shape=(len(disc_array),len(disc_array)))
#matrix_minus_K = np.zeros(shape=(len(disc_array),len(disc_array)))

#19
#new_df_pd_w_true_nan = new_df_pd.copy()

In [54]:
g1 = {3,5,8,9,10}
g2 = {3,5,8,9,10,7,4}
len(g2.difference(g1))

2